In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Euer erster Quantenschaltkreis

Baut's an [Bell-Zustand](https://en.wikipedia.org/wiki/Bell_state) (zwoa mitanond verschränkte Qubits) und führt's'n auf drei Arten aus:

1. **Ideale Simulation** — perfekte Ergebnisse, ka Konto notwendig
2. **Simulation mit Rauschen** — simuliert a echts Gerät, ka Konto notwendig
3. **Echte Quantenhardware** — braucht a [IBM Quantum-Konto](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Den Schaltkreis erstön

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Option 1: Ideale Simulation (ka Konto notwendig)
Verwendet `StatevectorSampler` — a lokaler Simulator mit perfekte, rauschfreie Ergebnisse.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Option 2: Simulation mit Rauschen (ka Konto notwendig)
Verwendet `FakeManilaV2` — a lokaler Simulator, der a echts IBM-Quantengerät nachahmt, a mit seine Rauscheigenschaften. Der Schaltkreis muss zuerst transpiliert (angepasst) wer'n, damit er zum Gate-Set und zur Qubit-Konnektivität vom Gerät passt.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Option 3: Echte Quantenhardware
Braucht a IBM Quantum-Konto. Schaut's bei [Setting up your IBM Quantum account](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) für de Details.

Wenn's eire Zugangsdaten noch net in der Binder-Sitzung g'speichert habt's, führt's des zuerst aus:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Achtung:** Jobs auf echter Hardware können a Zeitl dauern, je nach Wartezeit in da Warteschlange. Wenn de Zön no läuft, könnt's'n Job-Status und de Ergebnisse auf [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me) anschauen.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Was kommt als Nächstes?
- **[Tutorials](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — Schritt-für-Schritt-Anleitungen zu Algorithmen, Fehlerverminderung, Transpilation und mehr
- **[Courses](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — Strukturierte Lernpfade von Quantengrundlagen bis zu Utility-Scale-Computing
- **[Local testing mode](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — de meisten Notebooks ohne IBM Quantum-Konto ausführ'n